In [1]:
import pandas as pd
from glob import glob

In [2]:
df = pd.read_csv('data.csv')

In [3]:
X, y = df['img_path'], df['label']

In [10]:
import numpy as np
import cv2
from imblearn.over_sampling import SMOTE
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

# Hàm đọc hình ảnh từ đường dẫn
def read_image(image_path):
    return cv2.imread(image_path)

# Giả sử X là list chứa đường dẫn đến các frame và y là nhãn tương ứng
# X: List of strings (paths)
# y: numpy array of labels

# Bước 1: Áp dụng SMOTE
smote = SMOTE(random_state=42)

# Đọc hình ảnh và chuyển đổi thành feature vector
X_features = np.array([read_image(glob(os.path.join(path, '*.jpg'))).flatten() for path in X])

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object


In [12]:
import numpy as np
import cv2
from imblearn.over_sampling import SMOTE
import albumentations as A
import os
from glob import glob
from sklearn.preprocessing import LabelEncoder

# Hàm đọc và resize frame
def read_and_resize_frame(path, target_size=(224, 224)):
    frame = cv2.imread(path)
    if frame is not None:
        return cv2.resize(frame, target_size)
    return np.zeros((*target_size, 3), dtype=np.uint8)

# Hàm đọc tất cả các frame từ một thư mục
def read_frames_from_directory(directory, max_frames=None, target_size=(224, 224)):
    image_paths = sorted(glob(os.path.join(directory, '*')))
    if max_frames:
        image_paths = image_paths[:max_frames]
    frames = [read_and_resize_frame(path, target_size) for path in image_paths]
    return np.array(frames)

# Hàm chuẩn hóa số lượng frame
def normalize_frames(frames, target_frames):
    if len(frames) >= target_frames:
        return frames[:target_frames]
    else:
        padding = np.zeros((target_frames - len(frames), *frames.shape[1:]), dtype=frames.dtype)
        return np.vstack((frames, padding))

# Giả sử X là list chứa đường dẫn đến các thư mục chứa frame và y là nhãn tương ứng
# X: List of strings (paths to directories)
# y: numpy array of labels

# Xác định số lượng frame tối đa và kích thước frame
max_frames = max(len(glob(os.path.join(directory, '*'))) for directory in X)
target_size = (224, 224)  # Có thể điều chỉnh kích thước này

# Đọc và xử lý dữ liệu
X_processed = []
for directory in X:
    frames = read_frames_from_directory(directory, max_frames=max_frames, target_size=target_size)
    normalized_frames = normalize_frames(frames, max_frames)
    X_processed.append(normalized_frames)

X_processed = np.array(X_processed)

print("Shape của X_processed:", X_processed.shape)

# Reshape X_processed để SMOTE có thể xử lý
X_reshaped = X_processed.reshape(X_processed.shape[0], -1)

# Bước 1: Áp dụng SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_reshaped, y)

# Reshape lại sau khi áp dụng SMOTE
X_resampled = X_resampled.reshape(-1, max_frames, *target_size, 3)

# Bước 2: Áp dụng data augmentation cho lớp thiểu số (lớp 0)
augmentation = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussNoise(p=0.2),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.OneOf([
        A.OpticalDistortion(p=0.3),
        A.GridDistortion(p=0.1),
        A.ElasticTransform(p=0.1),
    ], p=0.2),
])

def augment_minority_class(X, y, num_augmentations=1):
    augmented_sequences = []
    augmented_labels = []
    
    for sequence, label in zip(X, y):
        if label == 0:  # Lớp thiểu số
            for _ in range(num_augmentations):
                augmented_frames = [augmentation(image=frame)['image'] for frame in sequence]
                augmented_sequences.append(np.array(augmented_frames))
                augmented_labels.append(label)
    
    return np.array(augmented_sequences), np.array(augmented_labels)

# Áp dụng data augmentation
X_aug, y_aug = augment_minority_class(X_resampled, y_resampled)

# Kết hợp dữ liệu gốc và dữ liệu đã augment
X_final = np.concatenate([X_resampled, X_aug])
y_final = np.concatenate([y_resampled, y_aug])

# Xáo trộn dữ liệu
indices = np.arange(len(X_final))
np.random.shuffle(indices)
X_final = X_final[indices]
y_final = y_final[indices]

print("Shape của X sau khi xử lý:", X_final.shape)
print("Shape của y sau khi xử lý:", y_final.shape)
print("Số lượng mẫu lớp 0:", np.sum(y_final == 0))
print("Số lượng mẫu lớp 1:", np.sum(y_final == 1))

# Lưu các chuỗi frame đã xử lý (tuỳ chọn)
output_dir = "processed_sequences"
os.makedirs(output_dir, exist_ok=True)

for i, sequence in enumerate(X_final):
    sequence_dir = os.path.join(output_dir, f"sequence_{i}")
    os.makedirs(sequence_dir, exist_ok=True)
    for j, frame in enumerate(sequence):
        cv2.imwrite(os.path.join(sequence_dir, f"frame_{j}.jpg"), frame)

Shape của X_processed: (111, 172, 224, 224, 3)
Shape của X sau khi xử lý: (249, 172, 224, 224, 3)
Shape của y sau khi xử lý: (249,)
Số lượng mẫu lớp 0: 166
Số lượng mẫu lớp 1: 83


In [13]:
X_processed

array([[[[[155, 155, 155],
          [153, 153, 153],
          [152, 152, 152],
          ...,
          [142, 142, 142],
          [139, 139, 139],
          [139, 139, 139]],

         [[155, 155, 155],
          [155, 155, 155],
          [152, 152, 152],
          ...,
          [138, 138, 138],
          [145, 145, 145],
          [139, 139, 139]],

         [[151, 151, 151],
          [154, 154, 154],
          [152, 152, 152],
          ...,
          [142, 142, 142],
          [135, 135, 135],
          [141, 141, 141]],

         ...,

         [[203, 203, 203],
          [209, 209, 209],
          [212, 212, 212],
          ...,
          [164, 164, 164],
          [164, 164, 164],
          [166, 166, 166]],

         [[194, 194, 194],
          [202, 202, 202],
          [208, 208, 208],
          ...,
          [166, 166, 166],
          [166, 166, 166],
          [164, 164, 164]],

         [[187, 187, 187],
          [194, 194, 194],
          [203, 203, 203],
         

In [14]:
import numpy as np
import cv2
from imblearn.over_sampling import SMOTE
import albumentations as A
import os
from glob import glob
from sklearn.preprocessing import LabelEncoder

# Hàm đọc và resize frame
def read_and_resize_frame(path, target_size=(224, 224)):
    frame = cv2.imread(path)
    if frame is not None:
        return cv2.resize(frame, target_size)
    return np.zeros((*target_size, 3), dtype=np.uint8)

# Hàm đọc tất cả các frame từ một thư mục
def read_frames_from_directory(directory, max_frames=None, target_size=(224, 224)):
    image_paths = sorted(glob(os.path.join(directory, '*')))
    if max_frames:
        image_paths = image_paths[:max_frames]
    frames = [read_and_resize_frame(path, target_size) for path in image_paths]
    return np.array(frames)

# Hàm chuẩn hóa số lượng frame
def normalize_frames(frames, target_frames):
    if len(frames) >= target_frames:
        return frames[:target_frames]
    else:
        padding = np.zeros((target_frames - len(frames), *frames.shape[1:]), dtype=frames.dtype)
        return np.vstack((frames, padding))

# Giả sử X là list chứa đường dẫn đến các thư mục chứa frame và y là nhãn tương ứng
# X: List of strings (paths to directories)
# y: numpy array of labels

# Xác định số lượng frame tối đa và kích thước frame
max_frames = max(len(glob(os.path.join(directory, '*'))) for directory in X)
target_size = (224, 224)  # Có thể điều chỉnh kích thước này

# Đọc và xử lý dữ liệu
X_processed = []
for directory in X:
    frames = read_frames_from_directory(directory, max_frames=max_frames, target_size=target_size)
    normalized_frames = normalize_frames(frames, max_frames)
    X_processed.append(normalized_frames)

X_processed = np.array(X_processed)

print("Shape của X_processed:", X_processed.shape)

# Reshape X_processed để SMOTE có thể xử lý
X_reshaped = X_processed.reshape(X_processed.shape[0], -1)

# Bước 1: Áp dụng SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_reshaped, y)

# Reshape lại sau khi áp dụng SMOTE
X_resampled = X_resampled.reshape(-1, max_frames, *target_size, 3)

# Bước 2: Áp dụng data augmentation cho lớp thiểu số (lớp 0)
augmentation = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussNoise(p=0.2),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.OneOf([
        A.OpticalDistortion(p=0.3),
        A.GridDistortion(p=0.1),
        A.ElasticTransform(p=0.1),
    ], p=0.2),
])

def augment_minority_class(X, y, num_augmentations=1):
    augmented_sequences = []
    augmented_labels = []
    
    for sequence, label in zip(X, y):
        if label == 0:  # Lớp thiểu số
            for _ in range(num_augmentations):
                augmented_frames = [augmentation(image=frame)['image'] for frame in sequence]
                augmented_sequences.append(np.array(augmented_frames))
                augmented_labels.append(label)
    
    return np.array(augmented_sequences), np.array(augmented_labels)

# Áp dụng data augmentation
X_aug, y_aug = augment_minority_class(X_resampled, y_resampled)

# Kết hợp dữ liệu gốc và dữ liệu đã augment
X_final = np.concatenate([X_resampled, X_aug])
y_final = np.concatenate([y_resampled, y_aug])

# Xáo trộn dữ liệu
indices = np.arange(len(X_final))
np.random.shuffle(indices)
X_final = X_final[indices]
y_final = y_final[indices]

print("Shape của X sau khi xử lý:", X_final.shape)
print("Shape của y sau khi xử lý:", y_final.shape)
print("Số lượng mẫu lớp 0:", np.sum(y_final == 0))
print("Số lượng mẫu lớp 1:", np.sum(y_final == 1))

# Lưu các chuỗi frame đã xử lý (tuỳ chọn)
output_dir = "processed_sequences"
os.makedirs(output_dir, exist_ok=True)

for i, (sequence, label) in enumerate(zip(X_final, y_final)):
    if i < len(X):  # Dữ liệu gốc
        subfolder = os.path.basename(os.path.dirname(X[i]))
    else:  # Dữ liệu được augment
        subfolder = f"augmented_{i - len(X)}"
    
    sequence_dir = os.path.join(output_dir, subfolder)
    os.makedirs(sequence_dir, exist_ok=True)
    
    for j, frame in enumerate(sequence):
        cv2.imwrite(os.path.join(sequence_dir, f"frame_{j}.jpg"), frame)
    
    # Lưu nhãn
    with open(os.path.join(sequence_dir, "label.txt"), "w") as f:
        f.write(str(label))

print(f"Đã lưu các chuỗi frame đã xử lý vào thư mục {output_dir}")

Shape của X_processed: (111, 172, 224, 224, 3)
Shape của X sau khi xử lý: (249, 172, 224, 224, 3)
Shape của y sau khi xử lý: (249,)
Số lượng mẫu lớp 0: 166
Số lượng mẫu lớp 1: 83
Đã lưu các chuỗi frame đã xử lý vào thư mục processed_sequences
